In [5]:
import requests
import pandas as pd

In [10]:
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"  # <-- increased the number of reviews to 9999

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()

    return data


r = requests.get(
    "https://www.vivino.com/api/explore/explore",
    params={
        "country_code": "FR",
        "country_codes[]": "pt",
        "currency_code": "EUR",
        "grape_filter": "varietal",
        "min_rating": "1",
        "order_by": "price",
        "order": "asc",
        "page": 1,
        "price_range_max": "500",
        "price_range_min": "400",
        "wine_type_ids[]": "1",
    },
    headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"
    },
)

results = [
    (
        t["vintage"]["wine"]["winery"]["name"],
        t["vintage"]["year"],
        t["vintage"]["wine"]["id"],
        f'{t["vintage"]["wine"]["name"]} {t["vintage"]["year"]}',
        t["vintage"]["statistics"]["ratings_average"],
        t["vintage"]["statistics"]["ratings_count"],
        t["prices"][0]["amount"],
        t["vintage"]["wine"]["style"]["acidity"],
        t["vintage"]["wine"]["style"]["blurb"],
        t["vintage"]["wine"]["style"]["body"],
        t["vintage"]["wine"]["style"]["body_description"],
        t['vintage']['wine']['region']['country']['name'],
        t['vintage']['wine']['style']['description'],
        t['vintage']['wine']['style']['food'][0]['name'],
        t['vintage']['wine']['style']['food'][1]['name'],
        t['vintage']['wine']['style']['food'][2]['name'],
        t['vintage']['wine']['style']['food'][3]['name'],
        t['vintage']['wine']['style']['country']['most_used_grapes'][0]['name'],
        t['vintage']['wine']['region']['country']['most_used_grapes'][1]['name'],
        t['vintage']['wine']['region']['country']['most_used_grapes'][2]['name'],
        t['vintage']['wine']['taste']['structure']['acidity'],
        t['vintage']['wine']['taste']['structure']['calculated_structure_count'],
        t['vintage']['wine']['taste']['structure']['intensity'],
        t['vintage']['wine']['taste']['structure']['sweetness'],
        t['vintage']['wine']['taste']['structure']['tannin'],
        t['vintage']['wine']['taste']['structure']['user_structure_count'],
        t['vintage']['wine']['taste']['flavor'][0]['group'],
        t['vintage']['wine']['taste']['flavor'][0]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][1]['group'],
        t['vintage']['wine']['taste']['flavor'][1]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][2]['group'],
        t['vintage']['wine']['taste']['flavor'][2]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][3]['group'],
        t['vintage']['wine']['taste']['flavor'][3]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][4]['group'],
        t['vintage']['wine']['taste']['flavor'][4]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][5]['group'],
        t['vintage']['wine']['taste']['flavor'][5]['stats']['count'],
        t['vintage']['wine']['taste']['flavor'][6]['group'],
        t['vintage']['wine']['taste']['flavor'][6]['stats']['count']
    )
    for t in r.json()["explore_vintage"]["matches"]
]
dataframe = pd.DataFrame(
    results,
    columns=["Winery", "Year", "Wine ID", "Wine", "Rating", "num_review", "price", "acidity",'Blurb','Body','Body_des','country','wine_des','food_1','food_2','food_3','food_4','grape_1','grape_2','grape_3','acidity_score','calculated_structure_count','intensity','sweetness','tannin','user_structure_count', 'flavor_1', 'flavor_1_count','flavor_2', 'flavor_2_count','flavor_3', 'flavor_3_count','flavor_4', 'flavor_4_count','flavor_5', 'flavor_5_count','flavor_6', 'flavor_6_count','flavor_7', 'flavor_7_count'],
             
)

ratings = []
for _, row in dataframe.iterrows():
    page = 1
    while True:
        print(
            f'Getting info about wine {row["Wine ID"]}-{row["Year"]} Page {page}'
        )

        d = get_wine_data(row["Wine ID"], row["Year"], page)

        if not d["reviews"]:
            break

        for r in d["reviews"]:
            ratings.append(
                [
                    row["Year"],
                    row["Wine ID"],
                    r["rating"],
                    r["note"],
                    r["created_at"],
                ]
            )

        page += 1

ratings = pd.DataFrame(
    ratings, columns=["Year", "Wine ID", "User Rating", "Note", "CreatedAt"]
)

df_out = ratings.merge(dataframe)
df_out.to_csv("data.csv", index=False)

Getting info about wine 2545450-2014 Page 1
Getting info about wine 2545450-2014 Page 2
Getting info about wine 2545450-2014 Page 3
Getting info about wine 2545450-2014 Page 4
Getting info about wine 2545450-2014 Page 5
Getting info about wine 2545450-2014 Page 6
Getting info about wine 2545450-2014 Page 7
Getting info about wine 2545450-2014 Page 8
Getting info about wine 2545450-2014 Page 9
Getting info about wine 2545450-2014 Page 10
Getting info about wine 2545450-2014 Page 11
Getting info about wine 2545450-2014 Page 12
Getting info about wine 2545450-2014 Page 13
Getting info about wine 2545450-2014 Page 14
Getting info about wine 2545450-2014 Page 15
Getting info about wine 2545450-2014 Page 16
Getting info about wine 2545450-2014 Page 17
Getting info about wine 2545450-2014 Page 18
Getting info about wine 2545450-2014 Page 19
Getting info about wine 2545450-2014 Page 20
Getting info about wine 2545450-2014 Page 21
Getting info about wine 2545450-2014 Page 22
Getting info about 

In [11]:
df_out.head()

,Year,Wine ID,User Rating,Note,CreatedAt,Winery,Wine,Rating,num_review,price,...,flavor_3,flavor_3_count,flavor_4,flavor_4_count,flavor_5,flavor_5_count,flavor_6,flavor_6_count,flavor_7,flavor_7_count
0,2014,2545450,5.0,World class wine and nothing much to say beyon...,2021-02-26T13:04:06.000Z,Cartuxa,Pêra-Manca Tinto 2014,4.7,1686,420,...,earth,60,non_oak,58,spices,38,red_fruit,26,microbio,13
1,2014,2545450,5.0,The Grand Master. The Petrus Alentejano. Simpl...,2021-07-15T11:11:36.000Z,Cartuxa,Pêra-Manca Tinto 2014,4.7,1686,420,...,earth,60,non_oak,58,spices,38,red_fruit,26,microbio,13
2,2014,2545450,5.0,"Vivino, eight years later 🍷\nBom Natal 🎄 \n",2020-12-26T23:17:42.000Z,Cartuxa,Pêra-Manca Tinto 2014,4.7,1686,420,...,earth,60,non_oak,58,spices,38,red_fruit,26,microbio,13
3,2014,2545450,5.0,"Excellent!4,8\nSunday barbecue at home...",2020-05-24T19:15:22.000Z,Cartuxa,Pêra-Manca Tinto 2014,4.7,1686,420,...,earth,60,non_oak,58,spices,38,red_fruit,26,microbio,13
4,2014,2545450,5.0,No comments my advice is to drink it,2020-10-29T15:20:44.000Z,Cartuxa,Pêra-Manca Tinto 2014,4.7,1686,420,...,earth,60,non_oak,58,spices,38,red_fruit,26,microbio,13


In [12]:
len(df_out)

5621